# IBM Data Science Professional Certificate Capstone Project:
## Analyzing Toronto's Boroughs Through Clustering
### This notebook is for the capstone project as part of a 9-course Data Science specialization offered by IBM through Coursera.

In [3]:
!conda install -c conda-forge beautifulsoup4 --yes

!conda install -c conda-forge geopy --yes

!conda install -c conda-forge folium=0.5.0 --yes

print('Libraries installed!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    beautifulsoup4-4.8.2       |           py36_0         157 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.8.2-py36_0          conda-forge
    ca-certificates: 2019.11.27-0      --> 2019.11.28-hecc5488_0 conda-forge
    certifi:         2019.11.28-py36_0 --> 2019.11.28-py36_0     conda-f

In [4]:
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize
import json

import requests

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported!')

Libraries imported!


In [5]:
print("Hello Capstone Poject Course")

Hello Capstone Poject Course


In [6]:
pd.__version__

'0.24.1'

#  PART 1: SEGMENTING AND CLUSTERING NEIGHBOURHOODS IN TORONTO
## Scraping the Wikipedia Page
### Using BeautifulSoup

In [7]:
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df_toronto = pd.read_html(str(table))[0]
postcode = df_toronto["Postcode"].tolist()
borough = df_toronto["Borough"].tolist()
neighbourhood = df_toronto["Neighbourhood"].tolist()
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Removing the 'Not assigned' cells in Borough

In [8]:
df_toronto["Borough"] = df_toronto["Borough"].replace("Not assigned", np.nan)
df_toronto.dropna(subset=["Borough"],inplace=True)
df_toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


## Combining Neighbourhood's that have the same Borough seperated by ','

In [9]:
df_toronto_grouped = df_toronto.groupby('Borough', as_index=False).agg(lambda x: ',' .join(x))
df_toronto_grouped.drop([5,7], axis=0, inplace=True)
df_toronto_grouped.drop('Postcode', axis=1, inplace=True)
df_toronto_grouped.reset_index(inplace=True)
df_toronto_grouped.drop('index', axis=1, inplace=True)
df_toronto_grouped

,Borough,Neighbourhood
0,Central Toronto,"Lawrence Park,Roselawn,Davisville North,Forest..."
1,Downtown Toronto,"Harbourfront,Queen's Park,Ryerson,Garden Distr..."
2,East Toronto,"The Beaches,The Danforth West,Riverdale,The Be..."
3,East York,"Woodbine Gardens,Parkview Hill,Woodbine Height..."
4,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar..."
5,North York,"Parkwoods,Victoria Village,Lawrence Heights,La..."
6,Scarborough,"Rouge,Malvern,Highland Creek,Rouge Hill,Port U..."
7,West Toronto,"Dovercourt Village,Dufferin,Little Portugal,Tr..."
8,York,"Humewood-Cedarvale,Caledonia-Fairbanks,Del Ray..."


## Shape of the cleaned DataFrame

In [10]:
df_toronto_grouped.shape

(9, 2)

## Getting coordinates and Avg. 1-Bedroom Apartment Price and adding them to the Toronto Grouped Dataframe
### Prices source: (https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart/Table?TableId=2.2.11&GeographyId=2270&GeographyTypeId=3&DisplayAs=Table&GeograghyName=Toronto)

In [11]:
data = {'Borough':  ['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto', 'Etobicoke', 'York', 'East York', 'Scarborough', 'North York'],
        'Latitude': [43.652503, 43.6808, 43.658315, 43.690773, 43.6205, 43.6957, 43.6912, 43.7764, 43.7615],
        'Longitude':[-79.383558, -79.2944, -79.442783, -79.403162, -79.5132, -79.4504, -79.3417, -79.2318, -79.4111]}

toronto_coords = pd.DataFrame (data, columns = ['Borough','Latitude', 'Longitude'])
print(toronto_coords.shape)
print(toronto_coords.head())

data = {'Borough':  ['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto', 'Etobicoke', 'York', 'East York', 'Scarborough', 'North York'],
        '1-Bedroom Apt. Avg. Price ($)': [1721, 1289, 1339, 1522, 1261, 1301, 1199, 1216, 1281]}
toronto_prices = pd.DataFrame (data, columns = ['Borough','1-Bedroom Apt. Avg. Price ($)'])
print(toronto_prices.shape)
print(toronto_prices.head())

(9, 3)
            Borough   Latitude  Longitude
0  Downtown Toronto  43.652503 -79.383558
1      East Toronto  43.680800 -79.294400
2      West Toronto  43.658315 -79.442783
3   Central Toronto  43.690773 -79.403162
4         Etobicoke  43.620500 -79.513200
(9, 2)
            Borough  1-Bedroom Apt. Avg. Price ($)
0  Downtown Toronto                           1721
1      East Toronto                           1289
2      West Toronto                           1339
3   Central Toronto                           1522
4         Etobicoke                           1261


## Merging the two DataFrames

In [12]:
df_toronto_temp = df_toronto_grouped.set_index('Borough')
coords_temp = toronto_coords.set_index('Borough')
df_toronto_coords = pd.concat([df_toronto_temp, coords_temp], axis=1, join='inner')

## Resetting the Index

In [13]:
df_toronto_coords.index.name = 'Borough'
df_toronto_coords.reset_index(inplace=True)

## Shape of merged coordinates Dataframe

In [14]:
print(df_toronto_coords.shape)
df_toronto_coords.head()

(9, 4)


,Borough,Neighbourhood,Latitude,Longitude
0,Central Toronto,"Lawrence Park,Roselawn,Davisville North,Forest...",43.690773,-79.403162
1,Downtown Toronto,"Harbourfront,Queen's Park,Ryerson,Garden Distr...",43.652503,-79.383558
2,East Toronto,"The Beaches,The Danforth West,Riverdale,The Be...",43.680800,-79.294400
3,East York,"Woodbine Gardens,Parkview Hill,Woodbine Height...",43.691200,-79.341700
4,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.620500,-79.513200


## Merging and creation of cleaned Toronto Dataframe

In [15]:
df_toronto_temp2 = df_toronto_coords.set_index('Borough')
coords_temp2 = toronto_prices.set_index('Borough')
df_toronto_clean = pd.concat([df_toronto_temp2, coords_temp2], axis=1, join='inner')
df_toronto_clean.drop('Neighbourhood', axis=1, inplace=True)
df_toronto_clean.reset_index(inplace=True)
df_toronto_clean

,Borough,Latitude,Longitude,1-Bedroom Apt. Avg. Price ($)
0,Central Toronto,43.690773,-79.403162,1522
1,Downtown Toronto,43.652503,-79.383558,1721
2,East Toronto,43.680800,-79.294400,1289
3,East York,43.691200,-79.341700,1199
4,Etobicoke,43.620500,-79.513200,1261
5,North York,43.761500,-79.411100,1281
6,Scarborough,43.776400,-79.231800,1216
7,West Toronto,43.658315,-79.442783,1339
8,York,43.695700,-79.450400,1301


# Part 2: Explore and cluster the boroughs in Toronto

In [16]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


## Using Folium to create a Map of Toronto with Boroughs markers on top

In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, borough, price in zip(df_toronto_clean['Latitude'], df_toronto_clean['Longitude'], df_toronto_clean['Borough'], df_toronto_clean['1-Bedroom Apt. Avg. Price ($)']):
    label = "{} (${})".format(borough, price)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

## Using FourSquare API to explore the Boroughs

In [18]:
CLIENT_ID = 'removed'
CLIENT_SECRET = 'removed'
VERSION = 'removed'

In [19]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, borough in zip(df_toronto_clean['Latitude'], df_toronto_clean['Longitude'], df_toronto_clean['Borough']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append(( 
            borough,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [31]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Borough', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(846, 7)


,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Central Toronto,43.690773,-79.403162,The Bagel House,43.687374,-79.393696,Bagel Shop
1,Central Toronto,43.690773,-79.403162,LCBO,43.686991,-79.399238,Liquor Store
2,Central Toronto,43.690773,-79.403162,Cava Restaurant,43.689809,-79.394932,Tapas Restaurant
3,Central Toronto,43.690773,-79.403162,Capocaccia Café,43.685915,-79.393305,Italian Restaurant
4,Central Toronto,43.690773,-79.403162,Scaramouche,43.681293,-79.399492,French Restaurant


## Check how many venues returned for each Borough

In [21]:
venues_df.groupby(['Borough'])['VenueName'].count()

Borough
Central Toronto     100
Downtown Toronto    100
East Toronto        100
East York           100
Etobicoke            92
North York          100
Scarborough          79
West Toronto        100
York                 75
Name: VenueName, dtype: int64

## Analyze venues in each borough

In [22]:
# one hot encoding
toronto_clean_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add borough column back to dataframe
toronto_clean_onehot['Borough'] = venues_df['Borough']  

# move borough column to the first column
fixed_columns = list(toronto_clean_onehot.columns[-1:]) + list(toronto_clean_onehot.columns[:-1])
toronto_clean_onehot = toronto_clean_onehot[fixed_columns]

print(toronto_clean_onehot.shape)
toronto_clean_onehot.head()

(846, 172)


,Borough,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Central Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Getting the frequency of occurance of each category in aa borough

In [23]:
toronto_clean_venues_freq = toronto_clean_onehot.groupby(['Borough']).mean().reset_index()
print(toronto_clean_venues_freq.shape)
toronto_clean_venues_freq.head()

(9, 172)


,Borough,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Central Toronto,0.00,0.030000,0.0,0.0,0.00,0.00,0.00000,0.01,0.020000,...,0.00,0.00,0.0,0.03,0.0,0.00000,0.0,0.01,0.01000,0.020000
1,Downtown Toronto,0.00,0.020000,0.0,0.0,0.02,0.01,0.01000,0.00,0.010000,...,0.01,0.00,0.0,0.02,0.0,0.00000,0.0,0.00,0.00000,0.020000
2,East Toronto,0.00,0.000000,0.0,0.0,0.00,0.00,0.00000,0.00,0.030000,...,0.01,0.00,0.0,0.01,0.0,0.00000,0.0,0.00,0.00000,0.000000
3,East York,0.01,0.020000,0.0,0.0,0.00,0.00,0.01000,0.00,0.010000,...,0.00,0.02,0.0,0.00,0.0,0.01000,0.0,0.00,0.00000,0.020000
4,Etobicoke,0.00,0.021739,0.0,0.0,0.00,0.00,0.01087,0.00,0.021739,...,0.00,0.00,0.0,0.00,0.0,0.01087,0.0,0.00,0.01087,0.021739


## Getting 10 most occurance venue types in each borough

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Borough']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns
# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = toronto_clean_venues_freq['Borough']

for ind in np.arange(toronto_clean_venues_freq.shape[0]):
    row_categories = toronto_clean_venues_freq.iloc[ind, :].iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    borough_venues_sorted.iloc[ind, 1:] = row_categories_sorted.index.values[0:num_top_venues]

borough_venues_sorted.sort_values(freqColumns, inplace=True)
borough_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,West Toronto,Café,Coffee Shop,Bar,Gastropub,Restaurant,Bakery,Cocktail Bar,Park,Brewery,Italian Restaurant
1,Downtown Toronto,Coffee Shop,Café,Hotel,Theater,Steakhouse,Gastropub,Pizza Place,Japanese Restaurant,Restaurant,Plaza
8,York,Coffee Shop,Grocery Store,Furniture / Home Store,Sandwich Place,Bakery,Pizza Place,Italian Restaurant,Fast Food Restaurant,Playground,Bank
6,Scarborough,Coffee Shop,Pharmacy,Sandwich Place,Gas Station,Park,Indian Restaurant,Fast Food Restaurant,Gym,Wings Joint,Chinese Restaurant
2,East Toronto,Coffee Shop,Pub,Bakery,Breakfast Spot,Park,Beach,BBQ Joint,Pharmacy,Thai Restaurant,Japanese Restaurant
4,Etobicoke,Coffee Shop,Restaurant,Italian Restaurant,Sandwich Place,Bakery,Convenience Store,Sushi Restaurant,Grocery Store,Fast Food Restaurant,Burrito Place
3,East York,Greek Restaurant,Café,Coffee Shop,Pizza Place,Gastropub,Ice Cream Shop,Bakery,Ethiopian Restaurant,Yoga Studio,Burger Joint
0,Central Toronto,Italian Restaurant,Park,Café,Coffee Shop,Restaurant,American Restaurant,Middle Eastern Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant,Indian Restaurant
5,North York,Korean Restaurant,Coffee Shop,Bubble Tea Shop,Pizza Place,Japanese Restaurant,Grocery Store,Ramen Restaurant,Bank,Sandwich Place,Fried Chicken Joint


## Clustering areas

In [63]:
kclusters = 3

toronto_clean_venues_freq_clustering = toronto_clean_venues_freq.drop(['Borough'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clean_venues_freq_clustering)

toronto_clean_clustered = df_toronto_clean  #toronto_central_df
toronto_clean_clustered['Cluster'] = kmeans.labels_

toronto_clean_clustered = toronto_clean_clustered.join(borough_venues_sorted.set_index('Borough'), on='Borough')
toronto_clean_clustered.sort_values(['Cluster'] + freqColumns, inplace=True)
toronto_clean_clustered

,Borough,Latitude,Longitude,1-Bedroom Apt. Avg. Price ($),Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,43.690773,-79.403162,1522,0,Italian Restaurant,Park,Café,Coffee Shop,Restaurant,American Restaurant,Middle Eastern Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant,Indian Restaurant
8,York,43.695700,-79.450400,1301,1,Coffee Shop,Grocery Store,Furniture / Home Store,Sandwich Place,Bakery,Pizza Place,Italian Restaurant,Fast Food Restaurant,Playground,Bank
6,Scarborough,43.776400,-79.231800,1216,1,Coffee Shop,Pharmacy,Sandwich Place,Gas Station,Park,Indian Restaurant,Fast Food Restaurant,Gym,Wings Joint,Chinese Restaurant
4,Etobicoke,43.620500,-79.513200,1261,1,Coffee Shop,Restaurant,Italian Restaurant,Sandwich Place,Bakery,Convenience Store,Sushi Restaurant,Grocery Store,Fast Food Restaurant,Burrito Place
5,North York,43.761500,-79.411100,1281,1,Korean Restaurant,Coffee Shop,Bubble Tea Shop,Pizza Place,Japanese Restaurant,Grocery Store,Ramen Restaurant,Bank,Sandwich Place,Fried Chicken Joint
7,West Toronto,43.658315,-79.442783,1339,2,Café,Coffee Shop,Bar,Gastropub,Restaurant,Bakery,Cocktail Bar,Park,Brewery,Italian Restaurant
1,Downtown Toronto,43.652503,-79.383558,1721,2,Coffee Shop,Café,Hotel,Theater,Steakhouse,Gastropub,Pizza Place,Japanese Restaurant,Restaurant,Plaza
2,East Toronto,43.680800,-79.294400,1289,2,Coffee Shop,Pub,Bakery,Breakfast Spot,Park,Beach,BBQ Joint,Pharmacy,Thai Restaurant,Japanese Restaurant
3,East York,43.691200,-79.341700,1199,2,Greek Restaurant,Café,Coffee Shop,Pizza Place,Gastropub,Ice Cream Shop,Bakery,Ethiopian Restaurant,Yoga Studio,Burger Joint


## Showing those clusters onto a map

In [65]:
# create map
map_clusters = folium.Map(width=2000,height=2000,location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, bor, price, cluster in zip(toronto_clean_clustered['Latitude'], toronto_clean_clustered['Longitude'], toronto_clean_clustered['Borough'], toronto_clean_clustered['1-Bedroom Apt. Avg. Price ($)'], toronto_clean_clustered['Cluster']):
    label = folium.Popup('{} (Avg Studio ${}): - Cluster {}'.format(bor, price, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Part 3: Results Analysis

In [60]:
df_toronto_final = toronto_clean_clustered
df_toronto_final.drop(['Latitude', 'Longitude'], axis=1, inplace=True)

In [61]:
df_toronto_final

,Borough,1-Bedroom Apt. Avg. Price ($),Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,1522,0,Italian Restaurant,Park,Café,Coffee Shop,Restaurant,American Restaurant,Middle Eastern Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant,Indian Restaurant
8,York,1301,1,Coffee Shop,Grocery Store,Furniture / Home Store,Sandwich Place,Bakery,Pizza Place,Italian Restaurant,Fast Food Restaurant,Playground,Bank
6,Scarborough,1216,1,Coffee Shop,Pharmacy,Sandwich Place,Gas Station,Park,Indian Restaurant,Fast Food Restaurant,Gym,Wings Joint,Chinese Restaurant
4,Etobicoke,1261,1,Coffee Shop,Restaurant,Italian Restaurant,Sandwich Place,Bakery,Convenience Store,Sushi Restaurant,Grocery Store,Fast Food Restaurant,Burrito Place
5,North York,1281,1,Korean Restaurant,Coffee Shop,Bubble Tea Shop,Pizza Place,Japanese Restaurant,Grocery Store,Ramen Restaurant,Bank,Sandwich Place,Fried Chicken Joint
7,West Toronto,1339,2,Café,Coffee Shop,Bar,Gastropub,Restaurant,Bakery,Cocktail Bar,Park,Brewery,Italian Restaurant
1,Downtown Toronto,1721,2,Coffee Shop,Café,Hotel,Theater,Steakhouse,Gastropub,Pizza Place,Japanese Restaurant,Restaurant,Plaza
2,East Toronto,1289,2,Coffee Shop,Pub,Bakery,Breakfast Spot,Park,Beach,BBQ Joint,Pharmacy,Thai Restaurant,Japanese Restaurant
3,East York,1199,2,Greek Restaurant,Café,Coffee Shop,Pizza Place,Gastropub,Ice Cream Shop,Bakery,Ethiopian Restaurant,Yoga Studio,Burger Joint


# Results:
## I. Cluster 0: (Mostly restaurants, parks and cafés)
### Central Toronto:
    As the most expensive borough in Toronto, it's positioning allows close access to all other boroughs but features mostly restaurants and parks. Therefore if you are looking for good meals and open air activities, Central Toronto would be a good option for you. 
## II. Cluster 1: (Mostly coffee shops, Grocery Stores and restaurants)
### York:
    York fits into the middle price range for studio accomodation and features useful everyday shops such as grocery stores and furniture/home stores. It also has a few selection of restaurants, banks and bakeries. York is also located close to Central and Downtown Toronto, making it the ideal Borough if you can not afford Central Toronto.  If you are looking for a reasonably affordable borough with a lot of convenient facilities, York might be for you. 
### Scarborough:
    Being the most expensive borough of Cluster 1, Scarborough is also located to the far east of Toronto, further away from Downtown than other boroughs. In the other hand Scarborough has more parks and gas stations than York, which is convenient if you own a car and enjoy the outdoors. With coffee shops being the most common venue, Scarborough trades the convenience of errand facilities for pharmacies and sandwich places. At last it also offers gyms and a few restaurant venues. 
## III. Cluster 2: (Mostly coffee shops, pubs/bars and cafés)
### Downtown Toronto:
    Downtown Toronto is the most expensive of all boroughs but offers the widest variety of venue choices. If you have a lot of visitors during the year they will be happy to know that this borough offers a wide veriety of Hotels. While at their stay they can enjoy leisure venues such as Theaters, Steakhouses, Gastropubs and restaurants. It is noteworthy to mention that Downtown Toronto does not have a wide veriety of grocery stores or parks. 
### East Toronto:
    East Toronto is significantly cheaper than Downtown Toronto but still has a central location. East Toronto features a lot of pubs and coffee shops, making it the night-life choice from the other boroughs. East Toronto is also features beaches and BBQ joints, which are not very frequent or absent in other boroughs. There is a good selection of bakeries and pharmacies and a decent selection of restaurants. In conclusion East Toronto makes a great central location for students on a medium budget 
